# 神经网络

可以使用 `torch.nn` 包来构建神经网络.

`nn` 包依赖于 `autograd `包来定义模型并对它们求导。一个 `nn.Module` 包含各个层和一个 `forward(input)` 方法，该方法返回 `output` 。

例如，下面这个神经网络可以对数字进行分类：

![](https://github.com/pchen12567/picture_store/blob/master/PyTorch/torch_02.jpg?raw=true)

这是一个简单的前馈神经网络（feed-forward network）。它接受一个输入，然后将它送入下一层，一层接一层的传递，最后给出输出。

一个神经网络的典型训练过程如下：

* 定义包含一些可学习参数（或者叫权重）的神经网络
* 在输入数据集上迭代
* 通过网络处理输入
* 计算损失（输出和正确答案的距离）
* 将梯度反向传播给网络的参数
* 更新网络的权重，一般使用一个简单的规则：`weight = weight - learning_rate * gradient`

## 定义一个神经网络

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class Net(nn.Module):
    
    def __init__(self):
        # nn.Module子类的函数必须在构造函数中执行父类的构造函数
        # 下式等价于nn.Module.__init__(self)
        super(Net, self).__init__()
        
        # 卷积层 '1'表示输入图片为单通道, '6'表示输出通道数，'5'表示卷积核为5*5
        self.conv1 = nn.Conv2d(1, 6, 5)
        # 卷积层
        self.conv2 = nn.Conv2d(6, 16, 5)
        
        # an affine operation: y = Wx + b
        # 仿射层/全连接层，y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # 卷积 -> 激活 -> 池化 
        # 2x2 Max pooling
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        
        # reshape，‘-1’表示自适应
        x = x.view(-1, self.num_flat_features(x))
        
        x = F.relu(self.fc1(x))
        
        x = F.relu(self.fc2(x))
        
        x = self.fc3(x)
        
        return x
    
    
    def num_flat_features(self, x):
        # 除去批大小维度的其余维度
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s  
        return num_features   

In [3]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


只需要定义 `forward` 函数，`backward` 函数会在使用 `autograd` 时自动定义，`backward` 函数用来计算导数。可以在 `forward` 函数中使用任何针对张量的操作和计算，还可以使用if、for循环、print、log等Python语法，写法和标准的Python写法一致。。

一个模型的可学习参数可以通过 `net.parameters()` 返回，`net.named_parameters` 可同时返回可学习的参数及名称。

In [4]:
params = list(net.parameters())
print(len(params))
print(params[0].size()) # conv1's .weight

10
torch.Size([6, 1, 5, 5])


In [5]:
for name,parameters in net.named_parameters():
    print(name,':',parameters.size())

conv1.weight : torch.Size([6, 1, 5, 5])
conv1.bias : torch.Size([6])
conv2.weight : torch.Size([16, 6, 5, 5])
conv2.bias : torch.Size([16])
fc1.weight : torch.Size([120, 400])
fc1.bias : torch.Size([120])
fc2.weight : torch.Size([84, 120])
fc2.bias : torch.Size([84])
fc3.weight : torch.Size([10, 84])
fc3.bias : torch.Size([10])


forward函数的输入和输出都是Tensor。

尝试一个随机的32x32的输入。注意，这个网络（LeNet）的期待输入是32x32。如果使用MNIST数据集来训练这个网络，要把图片大小重新调整到32x32。

In [6]:
in_put = torch.randn(1, 1, 32, 32)
out = net(in_put)
print(out)

tensor([[ 0.0491, -0.0959,  0.0618, -0.0923,  0.0409,  0.0294, -0.1684, -0.1712,
          0.1494,  0.0643]], grad_fn=<AddmmBackward>)


清零所有参数的梯度缓存，然后进行随机梯度的反向传播：

In [7]:
net.zero_grad()
out.backward(torch.randn(1, 10))

注意：

`torch.nn` 只支持小批量处理（mini-batches）。整个 `torch.nn` 包只支持小批量样本的输入，不支持单个样本。

比如，`nn.Conv2d` 接受一个4维的张量，即 `nSamples x nChannels x Height x Width`

如果是一个单独的样本，只需要使用 `input.unsqueeze(0)` 来添加一个“假的”批大小维度，将batch_size设为１。

在继续之前，回顾一下到目前为止看到的所有类。
* `torch.Tensor` - 一个多维数组，支持诸如 `backward()` 等的自动求导操作，同时也保存了张量的梯度。
* `nn.Module` - 神经网络模块。是一种方便封装参数的方式，具有将参数移动到GPU、导出、加载等功能。
* `nn.Parameter` - 张量的一种，当它作为一个属性分配给一个Module时，它会被自动注册为一个参数。
* `autograd.Function` - 实现了自动求导前向和反向传播的定义，每个Tensor至少创建一个Function节点，该节点连接到创建Tensor的函数并对其历史进行编码

## 损失函数

一个损失函数接受一对 `(output, target)` 作为输入，计算一个值来估计网络的输出和目标值相差多少。

output为网络的输出,target为实际值

`nn` 包中有很多不同的 `损失函数` 。
* `nn.MSELoss` 是比较简单的一种，它计算输出和目标的均方误差 `（mean-squared error）`。
* `nn.CrossEntropyLoss` 用来计算交叉熵损失。

In [8]:
output = net(in_put)
target = torch.randn(10) # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.9987, grad_fn=<MseLossBackward>)


现在，如果使用 `loss` 的 `.grad_fn` 属性跟踪反向传播过程，会看到计算图如下：

`input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
       -> view -> linear -> relu -> linear -> relu -> linear
       -> MSELoss
       -> loss`

所以，当调用 `loss.backward()` ，整张图开始关于loss微分，图中所有设置了 `requires_grad=True` 的张量的 `.grad` 属性累积着梯度张量。

向后跟踪几步：

In [9]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

## 反向传播

只需要调用 `loss.backward()` 来反向传播权重。需要清零现有的梯度，否则梯度将会与已有的梯度累加。

现在，将调用 `loss.backward()` ，并查看conv1层的偏置（bias）在反向传播前后的梯度。

In [10]:
# 运行.backward，观察调用之前和调用之后的grad
# 清零所有参数（parameter）的梯度缓存
net.zero_grad()

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0060,  0.0053,  0.0093,  0.0032,  0.0087, -0.0016])


## 更新权重

最简单的更新规则是随机梯度下降法（SGD）:

`weight = weight - learning_rate * gradient`

可以使用简单的python代码来实现:

In [11]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

然而，在使用神经网络时，可能希望使用各种不同的更新规则，如SGD、Nesterov-SGD、Adam、RMSProp等。为此，构建了一个较小的包 `torch.optim`，它实现了所有的这些方法。使用它很简单：

In [12]:
import torch.optim as optim

# 创建优化器（optimizer），指定要调整的参数和学习率
optimizer = optim.SGD(net.parameters(), lr=0.01)

# 在训练的迭代中：
# 先梯度清零(与net.zero_grad()效果一样)
optimizer.zero_grad()

# 计算损失
output = net(in_put)
loss = criterion(output, target)

#反向传播
loss.backward()

 # 更新参数
optimizer.step()